In [1]:
!pip install burr[start]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.9/342.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3

In [2]:
from typing import Tuple

from burr.core import action, State, ApplicationBuilder, serde

import numpy as np
import pandas as pd

In [3]:
@serde.serialize.register(pd.DataFrame)
def serialize_pandas_df(value: pd.DataFrame, pandas_kwargs: dict, **kwargs) -> dict:
    head = pandas_kwargs.get("head", 2)
    return {serde.KEY: "pandas.DataFrame", "data": value.head(head).to_dict()}

@serde.deserializer.register("pandas.DataFrame")
def deserialize_pandas_df(value: dict, **kwargs) -> pd.DataFrame:
    return pd.DataFrame.from_dict(value["data"])

In [4]:
@action(reads=[], writes=["step1"])
def act1(state: State, df: pd.DataFrame) -> State:
    return state.update(step1=df)

@action(reads=["step1"], writes=["step2"])
def act2(state: State) -> State:

    df = state['step1'].copy()
    df = df.T
    return state.update(step2=df)

@action(reads=["step2"], writes=[])
def end1(state: State) -> Tuple[dict, State]:

    df = state['step2'].copy()

    return (df.to_dict(), state)

@action(reads=["step2"], writes=[])
def end2(state: State) -> Tuple[pd.DataFrame, State]:

    df = state['step2'].copy()

    return (df, state)

In [5]:
def build_app1():
  app = (
        ApplicationBuilder()
        .with_actions(act1=act1, act2=act2, end=end1)
        .with_transitions(
            ("act1", "act2"),
            ("act2", "end"),
        )
        .with_tracker(
            project="test_pandas",
            params={"serde_kwargs": {"pandas_kwargs": {"head": 3}}},
        )
        .with_entrypoint("act1")
        .build()
    )
  return app

def build_app2():
  app = (
        ApplicationBuilder()
        .with_actions(act1=act1, act2=act2, end=end2)
        .with_transitions(
            ("act1", "act2"),
            ("act2", "end"),
        )
        .with_tracker(
            project="test_pandas2",
            params={"serde_kwargs": {"pandas_kwargs": {"head": 3}}},
        )
        .with_entrypoint("act1")
        .build()
    )
  return app

In [6]:
app1 = build_app1()

random_pandas_df = pd.DataFrame(np.random.rand(10, 5), columns=list("abcde"))

action, result, state = app1.run(halt_after=["end"], inputs={"df": random_pandas_df})

result

{0: {'a': 0.41832733396824484,
  'b': 0.3179518110656301,
  'c': 0.42376023998840806,
  'd': 0.5381964926945663,
  'e': 0.050327989427768394},
 1: {'a': 0.3974777354808011,
  'b': 0.9546079249737481,
  'c': 0.409054373883593,
  'd': 0.2415956934701089,
  'e': 0.24062876748843742},
 2: {'a': 0.664134571481792,
  'b': 0.9861922355766886,
  'c': 0.5538028163383563,
  'd': 0.24321763558581988,
  'e': 0.11209469650682835},
 3: {'a': 0.585649719719747,
  'b': 0.7478636373533538,
  'c': 0.4215678035849807,
  'd': 0.2660013722847775,
  'e': 0.2116448993977489},
 4: {'a': 0.8307284369785565,
  'b': 0.10901684795050126,
  'c': 0.5647836616793235,
  'd': 0.9653932214328832,
  'e': 0.23746232492663744},
 5: {'a': 0.36024415154488043,
  'b': 0.6900683879083628,
  'c': 0.9381118840736614,
  'd': 0.22891194826020833,
  'e': 0.09708523799132607},
 6: {'a': 0.5582217262504593,
  'b': 0.9686092599665698,
  'c': 0.6927431008558893,
  'd': 0.8651067963057757,
  'e': 0.8224184159730495},
 7: {'a': 0.254369

In [7]:
app2 = build_app2()

random_pandas_df = pd.DataFrame(np.random.rand(10, 5), columns=list("abcde"))

action, result, state = app2.run(halt_after=["end"], inputs={"df": random_pandas_df})

result

ERROR:burr.core.application:
********************************************************************************
-------------------------------------------------------------------
Oh no an error! Need help with Burr?
Join our discord and ask for help! https://discord.gg/4FxBMyzW5n
-------------------------------------------------------------------
> Action: `end` encountered an error!<
> State (at time of action):
{'__PRIOR_STEP': 'act2',
 '__SEQUENCE_ID': 2,
 'step1': '          a         b         c         d         ...',
 'step2': '          0         1         2         3         ...'}
> Inputs (at time of action):
{'df': '          a         b         c         d         ...'}
********************************************************************************
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/burr/core/application.py", line 793, in _step
    result, new_state = _run_single_step_action(
  File "/usr/local/lib/python3.10/dist-packages/bur

ValueError: Action end returned a non-dict result:           0         1         2         3         4         5         6  \
a  0.905238  0.073727  0.335074  0.970145  0.916383  0.270417  0.805456   
b  0.700716  0.164756  0.095050  0.211135  0.182162  0.954045  0.498872   
c  0.098668  0.066112  0.193812  0.009940  0.197822  0.150121  0.698508   
d  0.992351  0.553215  0.941634  0.003524  0.882315  0.587173  0.715479   
e  0.081968  0.334158  0.913319  0.777446  0.804380  0.749781  0.733637   

          7         8         9  
a  0.142867  0.878312  0.056244  
b  0.051732  0.053992  0.278968  
c  0.265285  0.187810  0.118702  
d  0.062860  0.390415  0.003058  
e  0.829198  0.358661  0.052239  . All results must be dictionaries.

In [9]:
from google.colab import output
output.serve_kernel_port_as_iframe(7241, cache_in_notebook=False, height='1000')

<IPython.core.display.Javascript object>